In [10]:
import os

import datasets
from dotenv import load_dotenv
from pprint import pp

load_dotenv()

gaia_benchmark = datasets.load_dataset(
    "gaia-benchmark/GAIA",
    '2023_all',
    cache_dir="../data/gaia-benchmark",
    token=os.getenv("HF_API_TOKEN"),
    trust_remote_code=True
)

print(f'GAIA loaded successfully.\n{gaia_benchmark}\nPreview: ')
pp(gaia_benchmark['validation'][0])
pp(gaia_benchmark['test'][0])

GAIA loaded successfully.
DatasetDict({
    test: Dataset({
        features: ['task_id', 'Question', 'Level', 'Final answer', 'file_name', 'file_path', 'Annotator Metadata'],
        num_rows: 301
    })
    validation: Dataset({
        features: ['task_id', 'Question', 'Level', 'Final answer', 'file_name', 'file_path', 'Annotator Metadata'],
        num_rows: 165
    })
})
Preview: 
{'task_id': 'c61d22de-5f6c-4958-a7f6-5e9707bd3466',
 'Question': 'A paper about AI regulation that was originally submitted to '
             'arXiv.org in June 2022 shows a figure with three axes, where '
             'each axis has a label word at both ends. Which of these words is '
             'used to describe a type of society in a Physics and Society '
             'article submitted to arXiv.org on August 11, 2016?',
 'Level': '2',
 'Final answer': 'egalitarian',
 'file_name': '',
 'file_path': '',
 'Annotator Metadata': {'Steps': '1. Go to arxiv.org and navigate to the '
                       